# 🏦 EBA 2025 Stress Test — Capital Preservation Strategy Session

Four banking specialists debate how to protect capital AND maximize profitability under the EBA 2025 adverse scenario.

**Session Objective:**
- Preserve capital under severe stress conditions (CET1 depletion ~370bps)
- Do NOT halt bank operations — maintain lending capacity
- Maximize profitability over the next 5 years
- **OUTPUT: Maximum 5 actionable strategic goals**

**Multi-Agent Architecture:**
- LLM instances attached to specialist characters
- Mix of **GPT-5.2** (OpenAI) and **Claude Opus** (Anthropic)
- A **Moderator** drives toward 5 concrete goals (summarizes, doesn't introduce ideas)
- Built with **LangGraph** for stateful orchestration

> Requires [OpenAI](https://platform.openai.com/) and [Anthropic](https://console.anthropic.com/) API keys.

In [1]:
!pip install openai anthropic langgraph -q

import os
from typing import TypedDict
from IPython.display import display, Markdown
from openai import OpenAI
from anthropic import Anthropic
from langgraph.graph import StateGraph, END

# API Keys (set in environment or replace here)
os.environ.setdefault("OPENAI_API_KEY", "YOUR_OPENAI_KEY_HERE")
os.environ.setdefault("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_KEY_HERE")

openai_client = OpenAI()
anthropic_client = Anthropic()


class LLM:
    """Wrapper for LLM instances with unified interface."""
    
    def __init__(self, provider: str, model: str, reasoning: str = "medium", verbosity: str = "medium"):
        self.provider = provider
        self.model = model
        self.reasoning = reasoning
        self.verbosity = verbosity
    
    def __repr__(self):
        return f"LLM({self.provider}/{self.model}, reasoning={self.reasoning}, verbosity={self.verbosity})"
    
    def call(self, system_prompt: str, user_input: str) -> str:
        """Call the LLM and return response text."""
        if self.provider == "openai":
            full_input = f"[SYSTEM]\n{system_prompt}\n\n[USER]\n{user_input}"
            response = openai_client.responses.create(
                model=self.model,
                input=full_input,
                reasoning={"effort": self.reasoning},
                text={"verbosity": self.verbosity},
            )
            return response.output_text
        
        elif self.provider == "anthropic":
            response = anthropic_client.messages.create(
                model=self.model,
                max_tokens=2048,
                system=system_prompt,
                messages=[{"role": "user", "content": user_input}],
            )
            return response.content[0].text
        
        else:
            raise ValueError(f"Unknown provider: {self.provider}")


# === Initialize LLM instances ===
LLMS = {
    "gpt5_high": LLM("openai", "gpt-5.2", reasoning="high", verbosity="medium"),
    "gpt5_medium": LLM("openai", "gpt-5.2", reasoning="medium", verbosity="high"),
    "gpt5_fast": LLM("openai", "gpt-5.2", reasoning="none", verbosity="low"),
    "opus": LLM("anthropic", "claude-opus-4-5"),
}


# === Graph State ===
class DebateState(TypedDict):
    transcript: list[dict]
    current_agenda_idx: int
    turns_this_agenda: int
    next_speaker: str
    stage_direction: str
    moderator_summary: str
    debate_complete: bool
    final_summary: str  # The 5 big goals

print("✅ Ready — LLMs initialized:")
for name, llm in LLMS.items():
    print(f"   {name}: {llm}")


[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Ready — LLMs initialized:
   gpt5_high: LLM(openai/gpt-5.2, reasoning=high, verbosity=medium)
   gpt5_medium: LLM(openai/gpt-5.2, reasoning=medium, verbosity=high)
   gpt5_fast: LLM(openai/gpt-5.2, reasoning=none, verbosity=low)
   opus: LLM(anthropic/claude-opus-4-20250514, reasoning=medium, verbosity=medium)


## 🎭 Characters, Scenario & Portfolio

**Participants:**
- 🛡️ **CRO** (Chief Risk Officer) — Credit risk, stress testing, balance sheet protection
- 💰 **CFO** (Chief Financial Officer) — Capital allocation, P&L, shareholder value
- ⚖️ **Head of Treasury** — Funding, liquidity, market access
- 📊 **Head of Credit** — Loan origination, underwriting, portfolio quality

**Moderator Role:**
- Summarizes what participants say (does NOT introduce new ideas)
- Pushes for specificity: "What exactly? By when? What's the capital impact?"
- Drives toward **5 actionable goals** by session end

In [2]:
CHARACTERS = {
    "margaret": {
        "name": "Margaret Holt",
        "title": "Chief Risk Officer",
        "emoji": "🛡️",
        "personality": (
            "Conservative, stress-test focused. Survived 2008. "
            "Protects the balance sheet but knows killing growth is also death."
        ),
        "llm": LLMS["gpt5_high"],
    },
    "daniel": {
        "name": "Daniel Ferreira",
        "title": "Chief Financial Officer",
        "emoji": "💰",
        "personality": (
            "P&L obsessed, shareholder value focused. "
            "Challenges anything without clear ROE impact. Pragmatic about trade-offs."
        ),
        "llm": LLMS["opus"],
    },
    "yuki": {
        "name": "Yuki Tanaka",
        "title": "Head of Treasury & Liquidity",
        "emoji": "⚖️",
        "personality": (
            "Funding and liquidity expert. Translates strategy into cash flow reality. "
            "The 'can we actually do this?' voice."
        ),
        "llm": LLMS["gpt5_medium"],
    },
    "rafael": {
        "name": "Rafael Mendes",
        "title": "Head of Credit & Lending",
        "emoji": "📊",
        "personality": (
            "Knows every sector's risk intimately. Protects client relationships "
            "but realistic about problem loans. Portfolio optimization expert."
        ),
        "llm": LLMS["gpt5_high"],
    },
    "moderator": {
        "name": "Moderator",
        "title": "Session Facilitator",
        "emoji": "🎙️",
        "personality": "Drives toward 5 actionable goals. Summarizes, never invents.",
        "llm": LLMS["gpt5_fast"],
    },
}

# === EBA 2025 ADVERSE SCENARIO ===
SCENARIO = """
## EBA 2025 STRESS TEST — ADVERSE SCENARIO

**Macroeconomic shock (3 years):**
- GDP: -6.3% cumulative
- Unemployment: +6 percentage points
- Commercial Real Estate: -30% prices
- Residential Real Estate: -16% prices
- Equity markets: -50% Year 1
- CET1 capital expected depletion: ~370bps

**Triggers:** Geopolitical escalation, trade wars, energy crisis, prolonged recession.

**Our Portfolio (EUR 447B):**
| Segment | Size | Risk |
|---------|------|------|
| Mortgages | EUR 196B (44%) | Moderate |
| Food & Agri | EUR 120B (27%) | Sector-dependent |
| Business Lending | EUR 71B (16%) | High cyclical |
| Leasing | EUR 39B (9%) | Moderate |
| Commercial RE | EUR 21B (4%) | CRITICAL |

NPL ratio: 1.7%

**SESSION GOAL:** 
Develop a 5-year strategy that preserves capital, maintains lending, AND maximizes profitability.
Output: Maximum 5 actionable goals.
"""

AGENDA = [
    "First Reactions — where are we most vulnerable?",
    "Portfolio Deep Dive — what to protect, reduce, or exit",
    "Capital vs. Growth — the hard trade-offs",
    "5-Year View — how do we emerge stronger?",
    "The 5 Big Goals — final consolidation",
]

MAX_TURNS_PER_AGENDA = 4  # Shorter discussions

# Display
for key, c in CHARACTERS.items():
    display(Markdown(f"**{c['emoji']} {c['name']}** — *{c['title']}*\n"))

display(Markdown("---\n**Agenda:** " + " → ".join(f"*{a.split('—')[0].strip()}*" for a in AGENDA)))

**🛡️ Margaret Holt** — *Chief Risk Officer*


**💰 Daniel Ferreira** — *Chief Financial Officer*


**⚖️ Yuki Tanaka** — *Head of Treasury & Liquidity*


**📊 Rafael Mendes** — *Head of Credit & Lending*


**🎙️ Moderator** — *Session Facilitator*


---
**Agenda:** *First Reactions* → *Portfolio Deep Dive* → *Capital vs. Growth* → *5-Year View* → *The 5 Big Goals*

## 🔧 System Prompts

Each character and the moderator gets their own system prompt.

In [3]:
def build_character_prompt(char_key: str) -> str:
    """Build a system prompt for a specific character."""
    char = CHARACTERS[char_key]
    other_chars = "\n".join(
        f"- {c['emoji']} {c['name']} ({c['title']})"
        for k, c in CHARACTERS.items() if k != char_key and k != "moderator"
    )
    
    return f"""You are {char['name']}, {char['title']} at a major European commercial bank.

PERSONALITY: {char['personality']}

OTHER PARTICIPANTS:
{other_chars}

SCENARIO:
{SCENARIO}

=== STRICT RULES ===

1. BE BRIEF. Maximum 3-4 sentences per response. No rambling.

2. IDEAS OVER NUMBERS. Focus on strategic ideas, not made-up percentages.
   BAD: "We should reduce exposure by 23.7% which saves 47bps..."
   GOOD: "We should exit the office CRE segment entirely — it's dead weight."

3. NO EMPTY STATEMENTS.
   BAD: "We need to be careful about risk."
   GOOD: "Exit office CRE. Protect agriculture — it's our franchise."

4. RESPOND TO OTHERS. Agree, disagree, or build on what they said.

5. ONE CONCRETE IDEA PER TURN. Don't try to solve everything at once.
"""


def build_moderator_prompt() -> str:
    """Build the system prompt for the moderator."""
    char_list = "\n".join(
        f"- {c['name']} (key: {k}): {c['title']}"
        for k, c in CHARACTERS.items() if k != "moderator"
    )
    agenda_list = "\n".join(f"{i+1}. {a}" for i, a in enumerate(AGENDA))
    
    return f"""You are the moderator of a banking strategy session.

=== YOUR PRIMARY GOAL ===
Guide this session toward an ACTIONABLE PLAN with MAXIMUM 5 BIG GOALS.

PARTICIPANTS:
{char_list}

AGENDA:
{agenda_list}

=== YOUR ROLE ===

1. SUMMARIZE, DON'T INTRODUCE — synthesize what participants say, never create new ideas

2. PUSH FOR CLARITY — "What exactly? Which segment? What's the trade-off?"

3. TRACK EMERGING GOALS — periodically note: "I'm hearing Goal 1: [X], Goal 2: [Y]..."

4. KEEP IT MOVING — if discussion stalls, move to next agenda item

=== RESPONSE FORMAT (strict JSON) ===
{{
    "next_speaker": "<margaret|daniel|yuki|rafael>",
    "stage_direction": "<brief, e.g., *nods*>",
    "moderator_summary": "<1-2 sentences summarizing emerging goals, or empty>",
    "move_to_next_agenda": false,
    "end_debate": false
}}

Respond ONLY with valid JSON.
"""


def build_final_summary_prompt() -> str:
    """Prompt for the moderator to produce the final 5 goals."""
    return """You are the moderator. The debate is complete.

Your task: Consolidate EVERYTHING discussed into EXACTLY 5 BIG GOALS.

FORMAT (use this exact structure):

## THE 5 BIG GOALS

**GOAL 1: [Short Title]**
What: [One sentence describing the action]
Why: [One sentence on the rationale from the discussion]
Timeline: [Year 1 / Years 2-3 / Years 4-5]

**GOAL 2: [Short Title]**
...

**GOAL 3: [Short Title]**
...

**GOAL 4: [Short Title]**
...

**GOAL 5: [Short Title]**
...

RULES:
- Extract goals FROM THE DISCUSSION — do not invent new ideas
- Each goal must be concrete and actionable
- Keep descriptions brief (1 sentence each)
- If participants disagreed, pick the approach with most support
- No fluff, no hedging
"""


MODERATOR_PROMPT = build_moderator_prompt()
FINAL_SUMMARY_PROMPT = build_final_summary_prompt()
print(f"✅ Prompts ready — shorter responses, ideas over numbers")

✅ Prompts ready — shorter responses, ideas over numbers


## 🔧 LangGraph Debate Engine

Define the nodes and build the state graph.

In [4]:
import json

def format_transcript(transcript: list[dict], current_agenda_idx: int) -> str:
    """Format the transcript for display to a character."""
    if not transcript:
        return f"[This is the start of the meeting. Current agenda: {AGENDA[current_agenda_idx]}]\n\nNo one has spoken yet. You are the first to speak."
    
    lines = [f"=== MEETING TRANSCRIPT ===\n[Current Agenda: {AGENDA[current_agenda_idx]}]\n"]
    for turn in transcript:
        char = CHARACTERS.get(turn["speaker"])
        if char:
            if turn.get("stage_direction"):
                lines.append(f"\n{turn['stage_direction']}\n")
            lines.append(f"**{char['emoji']} {char['name']}:** {turn['content']}\n")
        if turn.get("moderator_summary"):
            lines.append(f"\n🎙️ **Moderator:** {turn['moderator_summary']}\n")
    return "\n".join(lines)


def moderator_node(state: DebateState) -> dict:
    """Moderator picks the next speaker and provides stage direction."""
    transcript_text = format_transcript(state["transcript"], state["current_agenda_idx"])
    
    user_input = f"""{transcript_text}

Agenda: {AGENDA[state['current_agenda_idx']]} (item {state['current_agenda_idx'] + 1}/{len(AGENDA)})
Turns this item: {state['turns_this_agenda']}

Pick next speaker. Set move_to_next_agenda=true when topic is covered.
Set end_debate=true on the LAST agenda item when goals are clear."""
    
    response_text = CHARACTERS["moderator"]["llm"].call(
        system_prompt=MODERATOR_PROMPT,
        user_input=user_input
    )
    
    try:
        decision = json.loads(response_text)
    except json.JSONDecodeError:
        speakers = [k for k in CHARACTERS.keys() if k != "moderator"]
        idx = state["turns_this_agenda"] % len(speakers)
        decision = {
            "next_speaker": speakers[idx],
            "stage_direction": "",
            "moderator_summary": "",
            "move_to_next_agenda": state["turns_this_agenda"] >= MAX_TURNS_PER_AGENDA,
            "end_debate": False
        }
    
    new_agenda_idx = state["current_agenda_idx"]
    new_turns = state["turns_this_agenda"]
    debate_complete = decision.get("end_debate", False)
    
    if decision.get("move_to_next_agenda") and not debate_complete:
        new_agenda_idx = min(state["current_agenda_idx"] + 1, len(AGENDA) - 1)
        new_turns = 0
        # Auto-end on last agenda item after enough turns
        if new_agenda_idx >= len(AGENDA) - 1 and state["turns_this_agenda"] >= 4:
            debate_complete = True
    
    return {
        "next_speaker": decision.get("next_speaker", "margaret"),
        "stage_direction": decision.get("stage_direction", ""),
        "moderator_summary": decision.get("moderator_summary", ""),
        "current_agenda_idx": new_agenda_idx,
        "turns_this_agenda": new_turns,
        "debate_complete": debate_complete,
    }


def character_node(state: DebateState) -> dict:
    """A character speaks based on the transcript."""
    speaker_key = state["next_speaker"]
    char = CHARACTERS[speaker_key]
    
    transcript_text = format_transcript(state["transcript"], state["current_agenda_idx"])
    
    user_input = f"""{transcript_text}

YOUR TURN — {char['name']}
Agenda: {AGENDA[state['current_agenda_idx']]}

Be brief (3-4 sentences). One concrete idea. Respond to others."""
    
    system_prompt = build_character_prompt(speaker_key)
    response_text = char["llm"].call(system_prompt=system_prompt, user_input=user_input)
    
    new_turn = {
        "speaker": speaker_key,
        "content": response_text,
        "stage_direction": state.get("stage_direction", ""),
        "moderator_summary": state.get("moderator_summary", ""),
    }
    
    return {
        "transcript": state["transcript"] + [new_turn],
        "turns_this_agenda": state["turns_this_agenda"] + 1,
    }


def final_summary_node(state: DebateState) -> dict:
    """Generate the final 5 goals summary."""
    transcript_text = format_transcript(state["transcript"], state["current_agenda_idx"])
    
    user_input = f"""{transcript_text}

=== DEBATE COMPLETE ===

Now produce the final 5 BIG GOALS based on everything discussed above.
Extract from the discussion — do not invent."""
    
    # Use a more capable model for the summary
    summary = LLMS["opus"].call(
        system_prompt=FINAL_SUMMARY_PROMPT,
        user_input=user_input
    )
    
    return {"final_summary": summary}


def route_after_moderator(state: DebateState) -> str:
    """Route to character, final summary, or end."""
    if state.get("debate_complete"):
        return "final_summary"
    return "character"


# Build the graph
graph = StateGraph(DebateState)
graph.add_node("moderator", moderator_node)
graph.add_node("character", character_node)
graph.add_node("final_summary", final_summary_node)

graph.set_entry_point("moderator")
graph.add_conditional_edges("moderator", route_after_moderator, {
    "character": "character",
    "final_summary": "final_summary"
})
graph.add_edge("character", "moderator")
graph.add_edge("final_summary", END)

debate_graph = graph.compile()

print("✅ Graph compiled — includes final 5-goals summary")

✅ Graph compiled — includes final 5-goals summary


## 🚀 Run the Debate

Execute the graph and watch the debate unfold turn-by-turn.

In [5]:
# Initial state
initial_state: DebateState = {
    "transcript": [],
    "current_agenda_idx": 0,
    "turns_this_agenda": 0,
    "next_speaker": "",
    "stage_direction": "",
    "moderator_summary": "",
    "debate_complete": False,
    "final_summary": "",
}

print("🎬 Starting EBA 2025 Strategy Session...\n")
print("📋 GOAL: 5 actionable strategic goals\n")
display(Markdown(f"## {AGENDA[0].split('—')[0].strip()}\n*{AGENDA[0].split('—')[1].strip()}*\n"))

last_agenda_idx = 0
final_transcript = []
final_summary_text = ""

# Stream the debate
for state in debate_graph.stream(initial_state, {"recursion_limit": 100}):
    if "character" in state:
        node_state = state["character"]
        if node_state.get("transcript"):
            final_transcript = node_state["transcript"]
            latest_turn = final_transcript[-1]
            char = CHARACTERS[latest_turn["speaker"]]
            
            # Display moderator summary if present
            if latest_turn.get("moderator_summary"):
                display(Markdown(f"🎙️ *{latest_turn['moderator_summary']}*\n"))
            
            # Display stage direction
            if latest_turn.get("stage_direction"):
                display(Markdown(f"{latest_turn['stage_direction']}"))
            
            # Display character response
            display(Markdown(f"**{char['emoji']} {char['name']}:** {latest_turn['content']}\n"))
            display(Markdown("---"))
    
    elif "moderator" in state:
        node_state = state["moderator"]
        new_idx = node_state.get("current_agenda_idx", 0)
        if new_idx > last_agenda_idx:
            last_agenda_idx = new_idx
            display(Markdown(f"\n## {AGENDA[new_idx].split('—')[0].strip()}\n*{AGENDA[new_idx].split('—')[1].strip()}*\n"))
    
    elif "final_summary" in state:
        # Display the final 5 goals
        final_summary_text = state["final_summary"].get("final_summary", "")
        display(Markdown("\n---\n"))
        display(Markdown(final_summary_text))

print(f"\n✅ Session complete ({len(final_transcript)} turns)")

🎬 Starting EBA 2025 Strategy Session...

📋 GOAL: 5 actionable strategic goals



## First Reactions
*where are we most vulnerable?*


*opens session, looks to CRO*

**🛡️ Margaret Holt:** Our most acute vulnerability is Commercial Real Estate: it’s small in size but will dominate losses once prices gap down and refinancing shuts. One concrete move: impose an immediate stop on new office CRE lending (including extensions unless fully collateralized) and shift the team to early borrower triage—identify the “refinancing wall” cases now and push restructures before arrears start. If we don’t ringfence CRE immediately, it will leak into capital and management bandwidth for the next three years.


---

...

**💰 Daniel Ferreira:** **💰 Daniel Ferreira:** Agreed—Yuki's weekly liquidity gate keeps us honest and Margaret's exception firewall stops mission creep. Final consolidation: these 5 goals become our North Star with monthly Board scorecards tracking CRE exits, fortress clients signed, capital recycled, and liquidity headroom maintained. One last concrete move: freeze all new hiring except for the Capital Recycling Desk and mortgage specialists—every other function runs lean until we've executed Clean Slate and proven this model works.


---


---


## THE 5 BIG GOALS

**GOAL 1: Ringfence Commercial Real Estate**
What: Implement immediate stop on new office CRE lending/extensions (unless fully collateralized) and redeploy team to early refinancing-wall triage and restructures.
Why: Office CRE is the most acute vulnerability that will dominate losses and management bandwidth once prices gap down.
Timeline: Year 1

**GOAL 2: Execute Defensive Business Lending Strategy**
What: Gate all cyclical SME/corporate lending with no covenant-lite renewals, secured revolvers only, and push amortizing structures on refinancings.
Why: Cyclicals with thin margins and loose revolvers will blow up fast once unemployment jumps and demand drops.
Timeline: Year 1

**GOAL 3: Implement Liquidity Protection Program**
What: Launch contingent liquidity lockdown (reprice/reduce/exit low-quality committed facilities) and 90-day funding de-risk sprint (term out wholesale, pre-fund needs).
Why: Committed revolvers will get pulled as clients panic, potentially eating our HQLA just when markets shut.
Timeline: Year 1

**GOAL 4: Launch Prime Mortgage Offensive**
What: Aggressively push mortgage refinancing products for prime customers only, under recession overlay (pause high-LTV/BTL, tighter affordability), funded via stable sources.
Why: Mortgages are 44% of the book, rates will drop in recession, and housing recovers first—this preserves earnings power.
Timeline: Years 1-2

**GOAL 5: Create Capital Recycling Model**
What: Establish Capital Recycling Desk to make prime mortgages and fortress Food & Agri clients capital-light via covered bonds/RMBS and credit risk transfer.
Why: This enables continued lending through the downturn without letting CET1 depletion dictate strategy while competitors become zombies.
Timeline: Years 2-3


✅ Session complete (22 turns)
